# 21CSE558T - Deep Neural Network Architectures
## Module 5: Object Detection
### Notebook 01: Object Detection Introduction - Understanding the Problem

---

**Learning Objectives:**
- Understand the difference between classification, localization, and detection
- Recognize real-world applications of object detection
- Identify challenges in object detection tasks
- Learn the input/output format for detection systems

**Estimated Time:** 10 minutes

---

In [ ]:
# Cell 2: Setup and Imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

print("Environment setup complete!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

## What is Object Detection?

Object detection is a computer vision task that combines **classification** and **localization**. Let's understand the progression:

### The Three Computer Vision Tasks:

```
┌─────────────────────────────────────────────────────────────┐
│  1. IMAGE CLASSIFICATION                                    │
│     Question: "What is this?"                               │
│     Input:  Image                                           │
│     Output: Single label (e.g., "Cat")                      │
│                                                             │
│     [Image] → Neural Network → "Cat"                        │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  2. OBJECT LOCALIZATION                                     │
│     Question: "Where is THE object?"                        │
│     Input:  Image (with ONE primary object)                │
│     Output: Label + Bounding Box (x, y, w, h)               │
│                                                             │
│     [Image] → Neural Network → "Cat" + [100, 50, 200, 150]  │
└─────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────┐
│  3. OBJECT DETECTION                                        │
│     Question: "Where are ALL the objects?"                  │
│     Input:  Image (with MULTIPLE objects)                  │
│     Output: List of [Label + Bounding Box + Confidence]     │
│                                                             │
│     [Image] → Neural Network → [                            │
│         ("Cat", [100, 50, 200, 150], 0.95),                │
│         ("Dog", [300, 100, 180, 200], 0.87),               │
│         ("Cat", [450, 80, 150, 140], 0.92)                 │
│     ]                                                       │
└─────────────────────────────────────────────────────────────┘
```

**Key Differences:**
- **Classification**: Answers "what" (no location)
- **Localization**: Answers "what" + "where" (single object)
- **Detection**: Answers "what" + "where" for ALL objects (multiple objects)

In [ ]:
# Cell 4: Classification Example
# Create a simple synthetic image with a cat-like shape

def create_cat_image():
    """Create a simple cat representation"""
    img = np.ones((300, 400, 3), dtype=np.uint8) * 240  # Light background
    
    # Draw a simple cat silhouette (using rectangles and circles)
    # Body
    img[150:250, 150:300] = [255, 140, 0]  # Orange body
    # Head
    img[100:180, 180:270] = [255, 140, 0]  # Orange head
    # Ears
    img[80:120, 170:200] = [255, 140, 0]   # Left ear
    img[80:120, 250:280] = [255, 140, 0]   # Right ear
    # Eyes
    img[120:135, 200:215] = [0, 0, 0]      # Left eye
    img[120:135, 235:250] = [0, 0, 0]      # Right eye
    
    return img

# Create and display
cat_img = create_cat_image()

plt.figure(figsize=(8, 6))
plt.imshow(cat_img)
plt.axis('off')
plt.title('IMAGE CLASSIFICATION EXAMPLE', fontsize=16, fontweight='bold')
plt.text(200, 280, 'Output: "Cat"', 
         ha='center', va='center', 
         fontsize=20, fontweight='bold',
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.8))
plt.tight_layout()
plt.show()

print("Classification Output:")
print("  Label: Cat")
print("  (No location information)")

In [ ]:
# Cell 5: Localization Example
# Same image but with a bounding box around THE object

fig, ax = plt.subplots(1, 1, figsize=(8, 6))
ax.imshow(cat_img)
ax.axis('off')
ax.set_title('OBJECT LOCALIZATION EXAMPLE', fontsize=16, fontweight='bold')

# Draw bounding box around the cat
# Format: (x, y, width, height)
bbox_x, bbox_y, bbox_w, bbox_h = 140, 75, 150, 180
rect = patches.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h, 
                         linewidth=3, edgecolor='red', facecolor='none')
ax.add_patch(rect)

# Add label
ax.text(bbox_x + 5, bbox_y - 5, 'Cat', 
        fontsize=14, fontweight='bold', color='red',
        bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

print("Localization Output:")
print(f"  Label: Cat")
print(f"  Bounding Box: (x={bbox_x}, y={bbox_y}, w={bbox_w}, h={bbox_h})")
print("  (Single object with location)")

In [ ]:
# Cell 6: Detection Example - Multiple Objects
# Create an image with multiple objects

def create_multi_object_image():
    """Create image with multiple cats and dogs"""
    img = np.ones((400, 600, 3), dtype=np.uint8) * 240
    
    # Cat 1 (orange)
    img[80:160, 50:150] = [255, 140, 0]
    img[50:90, 60:100] = [255, 140, 0]  # head
    img[60:70, 70:85] = [0, 0, 0]  # eye
    
    # Dog 1 (brown)
    img[200:300, 100:220] = [139, 90, 43]
    img[180:230, 130:190] = [139, 90, 43]  # head
    img[190:200, 145:165] = [0, 0, 0]  # eye
    
    # Cat 2 (gray)
    img[120:200, 350:450] = [128, 128, 128]
    img[90:130, 370:430] = [128, 128, 128]  # head
    img[100:110, 385:400] = [0, 0, 0]  # eye
    
    # Dog 2 (light brown)
    img[250:340, 400:520] = [160, 120, 80]
    img[230:270, 430:490] = [160, 120, 80]  # head
    img[240:250, 450:465] = [0, 0, 0]  # eye
    
    return img

multi_img = create_multi_object_image()

# Detection results
detections = [
    {'label': 'Cat', 'bbox': (45, 45, 110, 120), 'confidence': 0.95, 'color': 'red'},
    {'label': 'Dog', 'bbox': (95, 175, 130, 130), 'confidence': 0.87, 'color': 'blue'},
    {'label': 'Cat', 'bbox': (345, 85, 110, 120), 'confidence': 0.92, 'color': 'green'},
    {'label': 'Dog', 'bbox': (395, 225, 130, 120), 'confidence': 0.89, 'color': 'purple'}
]

# Display with all bounding boxes
fig, ax = plt.subplots(1, 1, figsize=(12, 8))
ax.imshow(multi_img)
ax.axis('off')
ax.set_title('OBJECT DETECTION EXAMPLE - Multiple Objects', fontsize=16, fontweight='bold')

# Draw all bounding boxes
for det in detections:
    x, y, w, h = det['bbox']
    rect = patches.Rectangle((x, y), w, h, 
                             linewidth=3, edgecolor=det['color'], facecolor='none')
    ax.add_patch(rect)
    
    # Add label with confidence
    label_text = f"{det['label']} ({det['confidence']:.2f})"
    ax.text(x + 5, y - 5, label_text, 
            fontsize=12, fontweight='bold', color=det['color'],
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.9))

plt.tight_layout()
plt.show()

print("Object Detection Output:")
print("  List of detections:")
for i, det in enumerate(detections, 1):
    print(f"    {i}. Label: {det['label']}, BBox: {det['bbox']}, Confidence: {det['confidence']:.2f}")
print("\n  (Multiple objects, each with label + location + confidence)")

## Real-World Applications of Object Detection

Object detection is a foundational technology powering many modern AI systems:

### 1. **Autonomous Driving**
- Detect pedestrians, vehicles, traffic signs, lane markings
- Critical for safety: must work in real-time (30+ FPS)
- Example: Tesla Autopilot, Waymo self-driving cars

### 2. **Medical Imaging**
- Detect tumors, lesions, abnormalities in X-rays, CT scans, MRIs
- Assist radiologists in early disease detection
- Example: Cancer detection, organ segmentation

### 3. **Surveillance and Security**
- Person detection, face recognition, suspicious behavior detection
- Track multiple people across camera networks
- Example: Airport security, retail loss prevention

### 4. **Retail and E-Commerce**
- Product recognition on shelves (inventory management)
- Visual search ("find products similar to this image")
- Amazon Go stores: detect items picked up by customers

### 5. **Wildlife Monitoring**
- Detect and count animals in camera trap images
- Track endangered species populations
- Example: Conservation efforts, poaching prevention

### 6. **Agriculture**
- Detect crop diseases, pests, weeds
- Estimate crop yield by counting fruits/vegetables
- Example: Precision agriculture, automated harvesting robots

### 7. **Manufacturing Quality Control**
- Detect defects in products on assembly lines
- Real-time inspection at high speeds
- Example: Electronics manufacturing, automotive parts inspection

### 8. **Augmented Reality (AR)**
- Detect objects to overlay virtual information
- Example: Snapchat filters, IKEA furniture placement app

---

**Common Requirements:**
- **Accuracy**: Must minimize false positives and false negatives
- **Speed**: Many applications need real-time performance (30+ FPS)
- **Robustness**: Must work under various lighting, angles, occlusions

In [ ]:
# Cell 8: The Challenge - Difficult Cases
# Demonstrate challenging scenarios for object detection

def create_challenge_images():
    """Create images demonstrating detection challenges"""
    
    # 1. Occlusion - objects hiding each other
    occlusion = np.ones((200, 200, 3), dtype=np.uint8) * 240
    occlusion[80:180, 50:120] = [255, 140, 0]  # Front cat (orange)
    occlusion[100:180, 100:170] = [128, 128, 128]  # Back cat (gray, partially hidden)
    
    # 2. Small objects
    small_obj = np.ones((200, 200, 3), dtype=np.uint8) * 240
    small_obj[90:110, 90:110] = [255, 0, 0]  # Tiny object
    small_obj[140:160, 140:160] = [0, 255, 0]  # Another tiny object
    
    # 3. Crowded scene
    crowded = np.ones((200, 200, 3), dtype=np.uint8) * 240
    # Multiple overlapping objects
    crowded[50:100, 30:80] = [255, 0, 0]
    crowded[60:110, 70:120] = [0, 255, 0]
    crowded[70:120, 110:160] = [0, 0, 255]
    crowded[110:160, 40:90] = [255, 255, 0]
    crowded[120:170, 90:140] = [255, 0, 255]
    
    # 4. Different scales
    scales = np.ones((200, 200, 3), dtype=np.uint8) * 240
    scales[40:120, 30:110] = [255, 140, 0]  # Large object
    scales[140:170, 140:170] = [128, 128, 128]  # Small object
    
    # 5. Low lighting (darker image)
    low_light = np.ones((200, 200, 3), dtype=np.uint8) * 60  # Dark background
    low_light[70:150, 60:140] = [100, 80, 60]  # Barely visible object
    
    # 6. Rotation/Different angles
    rotation = np.ones((200, 200, 3), dtype=np.uint8) * 240
    # Simulate rotated object (diagonal rectangle)
    for i in range(50, 150):
        for j in range(max(0, i-20), min(200, i+20)):
            rotation[j, i] = [255, 140, 0]
    
    return occlusion, small_obj, crowded, scales, low_light, rotation

# Create challenge images
challenges = create_challenge_images()
titles = [
    '1. Occlusion\n(Objects hiding each other)',
    '2. Small Objects\n(Hard to detect)',
    '3. Crowded Scene\n(Many overlapping objects)',
    '4. Different Scales\n(Near vs Far)',
    '5. Low Lighting\n(Poor visibility)',
    '6. Rotation/Pose\n(Different angles)'
]

# Display all challenges
fig, axes = plt.subplots(2, 3, figsize=(14, 9))
fig.suptitle('OBJECT DETECTION CHALLENGES', fontsize=18, fontweight='bold')

for idx, (ax, img, title) in enumerate(zip(axes.flat, challenges, titles)):
    ax.imshow(img)
    ax.set_title(title, fontsize=11, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\nKey Challenges in Object Detection:")
print("  1. Occlusion: Objects partially hidden by other objects")
print("  2. Scale Variation: Objects at different distances (sizes)")
print("  3. Crowded Scenes: Many objects close together")
print("  4. Small Objects: Difficult to detect with limited pixels")
print("  5. Lighting Conditions: Shadows, darkness, glare")
print("  6. Viewpoint/Pose: Objects at different angles/orientations")
print("\nA robust detector must handle ALL these challenges!")

## Input vs Output Format

Understanding the data format is crucial for building object detection systems.

### INPUT Format
```python
# Input: An image
image = np.array(shape=(Height, Width, 3))  # RGB image
# Example: (640, 480, 3) for a 640x480 pixel image
```

### OUTPUT Format
```python
# Output: List of detections
detections = [
    {
        'class_name': 'cat',           # Object class (string)
        'class_id': 0,                 # Class ID (integer)
        'bbox': [x, y, width, height], # Bounding box coordinates
        'confidence': 0.95             # Confidence score (0 to 1)
    },
    {
        'class_name': 'dog',
        'class_id': 1,
        'bbox': [x2, y2, width2, height2],
        'confidence': 0.87
    },
    # ... more detections
]
```

### Bounding Box Coordinate Systems

There are multiple ways to represent bounding boxes:

1. **[x, y, width, height]** (XYWH format)
   - (x, y): Top-left corner coordinates
   - width, height: Box dimensions
   - Example: [100, 50, 200, 150]

2. **[x_min, y_min, x_max, y_max]** (XYXY format)
   - (x_min, y_min): Top-left corner
   - (x_max, y_max): Bottom-right corner
   - Example: [100, 50, 300, 200]

3. **[x_center, y_center, width, height]** (YOLO format)
   - (x_center, y_center): Center of the box
   - width, height: Box dimensions
   - Often normalized to [0, 1] range

### Example Output
```python
# Real detection output example
output = [
    {'class_name': 'person', 'bbox': [120, 80, 150, 320], 'confidence': 0.98},
    {'class_name': 'car', 'bbox': [350, 200, 280, 180], 'confidence': 0.92},
    {'class_name': 'bicycle', 'bbox': [50, 150, 100, 140], 'confidence': 0.85}
]
```

### Confidence Score
- Range: 0.0 to 1.0 (0% to 100%)
- Threshold: Typically filter detections with confidence < 0.5
- Higher confidence = more certain detection
- Example: confidence > 0.9 → very confident, confidence < 0.6 → uncertain

## Preview of Solutions

You might be wondering: **"How do we actually build an object detector?"**

Great question! This week (Week 13) we'll learn the **foundations** and building blocks. In Weeks 14-15, we'll implement complete detection systems.

---

### Week 13: **Foundations** (This Week)
Learn the core concepts needed for ANY object detection system:

1. **Bounding Boxes** - How to represent and manipulate boxes
2. **IoU (Intersection over Union)** - Measuring box overlap (critical metric)
3. **Non-Maximum Suppression (NMS)** - Removing duplicate detections
4. **Anchor Boxes** - Pre-defined box templates
5. **Evaluation Metrics** - mAP, precision, recall

**Think of it as:** Learning the alphabet before writing essays

---

### Week 14: **YOLO (You Only Look Once)**
**Approach:** Single-shot detector (fast and real-time)

**How it works:**
- Divide image into grid (e.g., 13×13)
- Each grid cell predicts bounding boxes + class probabilities
- Single pass through neural network → very fast (45+ FPS)

**Strengths:**
- ⚡ Real-time speed (great for video, autonomous driving)
- Simple architecture, end-to-end training

**Weaknesses:**
- Struggles with small objects
- Lower accuracy than two-stage methods

**Versions:** YOLO v1, v2, v3, v4, v5, v8 (constantly improving)

---

### Week 15: **R-CNN Family (Region-based CNN)**
**Approach:** Two-stage detector (accurate but slower)

**How it works:**
- **Stage 1:** Generate region proposals (where objects might be)
- **Stage 2:** Classify each region + refine bounding box

**Evolution:**
1. **R-CNN** (2014): Slow but groundbreaking
2. **Fast R-CNN** (2015): Faster training and inference
3. **Faster R-CNN** (2016): Real-time region proposals (RPN)
4. **Mask R-CNN** (2017): Adds instance segmentation (pixel-level masks)

**Strengths:**
- 🎯 High accuracy (better than YOLO on small objects)
- Precise bounding boxes

**Weaknesses:**
- Slower than YOLO (not suitable for real-time in some cases)
- More complex architecture

---

### Comparison Table

| Aspect | YOLO (Week 14) | R-CNN Family (Week 15) |
|--------|---------------|------------------------|
| **Speed** | Very Fast (45+ FPS) | Moderate (5-15 FPS) |
| **Accuracy** | Good | Excellent |
| **Stages** | Single-stage | Two-stage |
| **Small Objects** | Struggles | Handles well |
| **Use Case** | Real-time apps | High-accuracy needs |
| **Complexity** | Simpler | More complex |

---

### Modern Trends (Beyond this course)
- **EfficientDet**: Balances speed and accuracy
- **Vision Transformers (ViT)**: Attention-based detectors (DETR)
- **Mobile Detectors**: MobileNet-SSD, Tiny YOLO (for phones/edge devices)

---

**This Week's Goal:** Master the foundational concepts so you can understand and implement BOTH YOLO and R-CNN confidently!

## Exercise

Test your understanding with these questions:

---

### TODO Exercise 1: Task Classification

**Scenario:** You have an image containing 3 cats and 2 dogs. You want to identify ALL animals and their locations.

**Question:** Is this a:
- A) Classification task?
- B) Localization task?
- C) Detection task?

**Your Answer:** ___ (Write A, B, or C)

**Explanation:** (Write 1-2 sentences explaining why)

```
YOUR ANSWER HERE:



```

---

### TODO Exercise 2: Real-World Applications

**Question:** List 3 real-world applications where object detection is **critical** (not just helpful, but essential).

For each application, explain in 1 sentence WHY object detection is critical.

**Your Answers:**
```
1. Application: _____________
   Why critical: _____________

2. Application: _____________
   Why critical: _____________

3. Application: _____________
   Why critical: _____________
```

---

### TODO Exercise 3: Challenges Analysis

**Question:** What makes object detection harder than simple image classification?

List at least 3 specific reasons with brief explanations.

**Your Answers:**
```
1. _____________

2. _____________

3. _____________

(Optional) 4. _____________
```

---

### TODO Exercise 4: Output Format

**Question:** Given this detection output:
```python
detection = {
    'class_name': 'person',
    'bbox': [100, 50, 200, 300],
    'confidence': 0.87
}
```

**Questions:**
- a) What are the coordinates of the top-left corner of the bounding box?
- b) What is the width and height of the bounding box?
- c) How confident is the model about this detection (in percentage)?
- d) If we use a confidence threshold of 0.9, will this detection be kept or filtered out?

**Your Answers:**
```
a) Top-left corner: _____________

b) Width: _____, Height: _____

c) Confidence: _____%

d) Kept or filtered? _____ (explain why)
```

---

**Check your answers with your instructor or peers before moving to the next notebook!**

## Summary

Congratulations! You've completed the introduction to object detection.

### Key Takeaways

1. **Three Computer Vision Tasks:**
   - Classification: "What is this?" → Single label
   - Localization: "Where is THE object?" → Label + 1 bounding box
   - Detection: "Where are ALL objects?" → Multiple [label + box + confidence]

2. **Real-World Impact:**
   - Object detection powers autonomous vehicles, medical diagnosis, security systems, retail automation, and many more applications
   - Requires both accuracy and speed for practical deployment

3. **Key Challenges:**
   - Occlusion (objects hiding each other)
   - Scale variation (small and large objects)
   - Crowded scenes with overlapping objects
   - Different lighting and viewing angles

4. **Input/Output Format:**
   - Input: Image (H × W × 3)
   - Output: List of [class, bounding box, confidence score]
   - Multiple coordinate systems: XYWH, XYXY, center-based

5. **Learning Path:**
   - Week 13: Foundations (IoU, NMS, anchors, metrics) ← You are here
   - Week 14: YOLO (fast, single-stage detector)
   - Week 15: R-CNN (accurate, two-stage detector)

---

### What's Next?

**Next Notebook:** `02_iou_calculation.ipynb`

You'll learn about **IoU (Intersection over Union)**, the most important metric for:
- Measuring how well a predicted box matches the ground truth
- Deciding which detections to keep (Non-Maximum Suppression)
- Evaluating detector performance (mAP calculation)

**Preview:** We'll implement IoU from scratch and visualize box overlaps!

---

### Before You Go

✅ Make sure you understand the difference between classification, localization, and detection  
✅ Complete all TODO exercises above  
✅ Review the bounding box coordinate formats  
✅ Think about a real-world application you'd like to build  

---

**Questions?** Ask your instructor or discuss with peers in the next class!

**Ready?** Move on to Notebook 02: IoU Calculation